In [106]:
import sys
import numpy as np
import pandas as pd
from helpers import *
import matplotlib.pyplot as plt
%matplotlib notebook


In [107]:
filename_measures = 'data/IMPROVE_2015_measures_cs433.csv'
filename_spectra = 'data/IMPROVE_2015_raw_spectra_cs433.csv'
filename_tts = 'data/IMPROVE_2015_train_test_split_cs433.csv'
# filename_sec_deriv = 'data/IMPROVE_2015_2nd-derivative_spectra_cs433.csv'

df_spectra_raw = pd.read_csv(filename_spectra)
df_measures_raw = pd.read_csv(filename_measures)
df_train_test_split_raw = pd.read_csv(filename_tts)
# df_second_derivative = pd.read_csv(filename_sec_deriv, index_col=0)

In [108]:
filename_measures = 'IMPROVE_2015_measures_cs433.csv'
df_meas = pd.read_csv(filename_measures)
dustValues = df_meas['DUSTf:Value']
dustValues = dustValues[~(np.isnan(dustValues))]

### Load data

In [109]:
df_spectra = df_spectra_raw.T
# df_spectra.columns = df_spectra.loc['wavenumber',:]
df_spectra.columns = pd.Float64Index(df_spectra.loc['wavenumber',:], name="")
df_spectra = df_spectra.drop('wavenumber')
#df_spectra

In [110]:
meta_cols = ['SiteCode','Date','flag','Latitude','Longitude','DUSTf:Unc']
y_col = ['DUSTf:Value']
df_measures = df_measures_raw.set_index('site')
df_measures = df_measures[meta_cols + y_col]
df_measures.index = pd.Index(df_measures.index, name="")
merged = pd.merge(df_spectra, df_measures, left_index=True, right_index=True)

## remove nan

In [111]:
nan_indices = merged['DUSTf:Value'].index[merged['DUSTf:Value'].apply(np.isnan)]
merged.drop(nan_indices, inplace=True)


## X,y creation

In [112]:
X = merged.loc[:, [x for x in merged.columns if x not in y_col and x not in meta_cols]]
y = merged[y_col]

# Cross validation

###### build_k_indices will help to divide the X into 10 partitions, then shuffle it.

In [113]:
def build_k_indices(num_row,k_fold, seed):
    """build k indices for k-fold."""
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

##### compute mse

In [114]:
def compute_mse(y, tx, w):
    """compute the loss by mse."""
    e = y - tx.dot(w)
    mse = e.T @ e/ (2 * y.shape[0])
    return mse


#####  ridge regression

In [115]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    lambda_prime = 2 * y.shape[0] * lambda_
    A = tx.T.dot(tx) + lambda_prime * np.eye(tx.shape[1])
    b = tx.T.dot(y)
    w = np.linalg.solve(A, b)
    return w

#### build polynomial

In [116]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

#### cross validation

In [120]:
def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train
    te_indice = k_indices[k] ## Choose one partition as test set
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    
    y_te = y.iloc[te_indice].values
    y_tr = y.iloc[np.array(tr_indice)].values
    x_te = x.iloc[te_indice].values
    x_tr = x.iloc[tr_indice].values
    # form data with polynomial degree
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)
    # ridge regression
    w = ridge_regression(y_tr, x_tr, lambda_)
    # calculate the loss for train and test data
    loss_tr = np.sqrt(2 *compute_mse(y_tr, x_tr, w))
    loss_te = np.sqrt(2 * compute_mse(y_te, x_te, w))
    return loss_tr, loss_te,w

###### demo

##### cross_validation_visualization

In [121]:
def cross_validation_visualization(lambds, mse_tr, mse_te):
    """visualization the curves of mse_tr and mse_te."""
    plt.semilogx(lambds, mse_tr, marker=".", color='b', label='train error')
    plt.semilogx(lambds, mse_te, marker=".", color='r', label='test error')
    plt.xlabel("lambda")
    plt.ylabel("rmse")
    plt.title("cross validation")
    plt.legend(loc=2)
    plt.grid(True)
    plt.savefig("cross_validation")

In [123]:
def cross_validation_demo():
    seed = 12
    degree = 1
    k_fold = 10
    lambdas = np.logspace(-4, 0, 30)
    # split data in k fold
    num_row = y.shape[0]
    k_indices = build_k_indices(num_row, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    # cross validation
    for ind, lambda_ in enumerate(lambdas):
        rmse_tr_tmp = []
        rmse_te_tmp = []
        for k in range(k_fold):
            loss_tr, loss_te,_ = cross_validation(y, X, k_indices, k, lambda_, degree)
            rmse_tr_tmp.append(loss_tr)            
            rmse_te_tmp.append(loss_te)
        rmse_tr.append(np.mean(rmse_tr_tmp))
        rmse_te.append(np.mean(rmse_te_tmp))
        print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
            p=k_fold, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))
    cross_validation_visualization(lambdas, rmse_tr, rmse_te)

cross_validation_demo()

proportion=10, degree=2, lambda=0.000, Training RMSE=1.875, Testing RMSE=1.856
proportion=10, degree=2, lambda=0.000, Training RMSE=1.911, Testing RMSE=1.892
proportion=10, degree=2, lambda=0.000, Training RMSE=1.950, Testing RMSE=1.932
proportion=10, degree=2, lambda=0.000, Training RMSE=1.993, Testing RMSE=1.977
proportion=10, degree=2, lambda=0.000, Training RMSE=2.044, Testing RMSE=2.030
proportion=10, degree=2, lambda=0.000, Training RMSE=2.105, Testing RMSE=2.092
proportion=10, degree=2, lambda=0.001, Training RMSE=2.178, Testing RMSE=2.169
proportion=10, degree=2, lambda=0.001, Training RMSE=2.266, Testing RMSE=2.260
proportion=10, degree=2, lambda=0.001, Training RMSE=2.373, Testing RMSE=2.371
proportion=10, degree=2, lambda=0.002, Training RMSE=2.501, Testing RMSE=2.501
proportion=10, degree=2, lambda=0.002, Training RMSE=2.653, Testing RMSE=2.656
proportion=10, degree=2, lambda=0.003, Training RMSE=2.833, Testing RMSE=2.837
proportion=10, degree=2, lambda=0.005, Training RMSE

KeyboardInterrupt: 